# 测试视频播放器

In [1]:
import tkinter as tk
from tkinter import ttk
import cv2
from PIL import Image, ImageTk
class VideoPlayer:
    def __init__(self, root,):
        self.root = root
        self.root.title("视频播放器")

        # 创建一个Label用于显示视频帧
        self.label = ttk.Label(root)
        self.label.pack()

        # 打开视频文件
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("无法打开视频文件")
            return

        # 开始更新视频帧
        self.update_frame()

    def update_frame(self):
        # 读取一帧
        ret, frame = self.cap.read()
        if ret:
            # 将BGR格式转换为RGB格式
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 将帧转换为PIL图像
            im = Image.fromarray(frame)
            # 将PIL图像转换为Tkinter兼容的PhotoImage
            img = ImageTk.PhotoImage(image=im)

            # 更新Label的图像
            self.label.config(image=img)
            self.label.image = img  # 避免被垃圾回收

            # 每隔30ms更新一次帧
            self.root.after(30, self.update_frame)
        else:
            # 如果视频结束，释放资源
            self.cap.release()
            print("视频播放结束")

root = tk.Tk()
player = VideoPlayer(root)
root.mainloop()

NameError: name 'cv2' is not defined

# 测试视频写入

In [6]:
# !pip install pyaudio  moviepy

  Using cached PyAudio-0.2.14-cp310-cp310-win_amd64.whl.metadata (2.7 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached pillow-10.4.0-cp310-cp310-win_amd64.whl.metadata (9.3 kB)
Using cached PyAudio-0.2.14-cp310-cp310-win_amd64.whl (164 kB)
   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/31.2 MB ? eta -:--:--
    --------------------------------------- 0.5/31.2 MB 932.9 kB/s eta 0:00:33
    --------------------------------------- 0.5/31.2 MB 932.9 kB/s eta 0:00:33
   - -------------------------------------- 0.8/31.2 MB 932.9 kB/s eta 0:00:33
   - -------------------------------------- 1.0/31.2 MB 968.5 kB/s eta 0:00:32
   - -------------------------------------- 1.3/31.2 MB 972.7 kB/s eta 0:00:31
   - -------------------------------------- 1.3/31.2 MB 972.7 kB/s eta 0:00:31
   -- ------------------------------------- 1.6/31.2 MB 953.2 kB/s eta 0:00:32
   -- --------------------------

In [ ]:
import cv2
import pyaudio
import wave
from moviepy import VideoFileClip, AudioFileClip
import threading

# 视频参数
video_output = 'video.avi'
video_fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_fps = 30.0

# 音频参数
audio_output = 'audio.wav'
audio_format = pyaudio.paInt16
audio_channels = 1
audio_rate = 44100
audio_chunk = 1024

# 初始化摄像头
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("无法打开摄像头")
    exit()

# 获取摄像头分辨率
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 创建视频写入对象
video_writer = cv2.VideoWriter(video_output, video_fourcc, video_fps, (frame_width, frame_height))

# 初始化音频录制
audio = pyaudio.PyAudio()
audio_stream = audio.open(format=audio_format, channels=audio_channels, rate=audio_rate, input=True, frames_per_buffer=audio_chunk)

# 用于存储音频数据
audio_frames = []

# 视频录制函数
def record_video():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        video_writer.write(frame)
        cv2.imshow('Video Capture', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# 音频录制函数
def record_audio():
    while True:
        data = audio_stream.read(audio_chunk)
        audio_frames.append(data)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# 启动音频录制线程
audio_thread = threading.Thread(target=record_audio)
audio_thread.start()

# 启动视频录制
record_video()

# 等待音频线程结束
audio_thread.join()

# 释放资源
cap.release()
video_writer.release()
audio_stream.stop_stream()
audio_stream.close()
audio.terminate()
cv2.destroyAllWindows()

# 保存音频文件
with wave.open(audio_output, 'wb') as wf:
    wf.setnchannels(audio_channels)
    wf.setsampwidth(audio.get_sample_size(audio_format))
    wf.setframerate(audio_rate)
    wf.writeframes(b''.join(audio_frames))

# 合并视频和音频
video_clip = VideoFileClip(video_output)
audio_clip = AudioFileClip(audio_output)
video_clip = video_clip.set_audio(audio_clip)
video_clip.write_videofile("output_with_audio.mp4", codec="libx264")

print("视频和音频合并完成，输出文件为 output_with_audio.mp4")

In [2]:
import cv2
import pyaudio
import wave
import threading
from datetime import datetime

# 视频参数
CAMERA_INDEX = 0  # 摄像头设备号
FRAME_RATE = 30
RESOLUTION = (1280, 720)
VIDEO_FILENAME = "temp_video.avi"

# 音频参数
AUDIO_FILENAME = "temp_audio.wav"
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
SAMPLE_RATE = 44100

# 全局控制变量
recording = True

def record_video():
    cap = cv2.VideoCapture(CAMERA_INDEX)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(VIDEO_FILENAME, fourcc, FRAME_RATE, RESOLUTION)
    
    while recording:
        ret, frame = cap.read()
        if ret:
            out.write(frame)
    cap.release()
    out.release()

def record_audio():
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=SAMPLE_RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    frames = []
    
    while recording:
        data = stream.read(CHUNK)
        frames.append(data)
    
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    # 保存音频
    wf = wave.open(AUDIO_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(SAMPLE_RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

# 启动线程
video_thread = threading.Thread(target=record_video)
audio_thread = threading.Thread(target=record_audio)
video_thread.start()
audio_thread.start()

# 按Enter键停止录制
input("按Enter键停止录制...")
recording = False
video_thread.join()
audio_thread.join()

# 合并音视频
from moviepy import VideoFileClip, AudioFileClip
video_clip = VideoFileClip(VIDEO_FILENAME)
audio_clip = AudioFileClip(AUDIO_FILENAME)
final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile("output_with_audio.mp4", codec="libx264", audio_codec="aac")


ModuleNotFoundError: No module named 'moviepy.editor'

# 测试主程序

In [ ]:
import tkinter as tk
from http import HTTPStatus
from tkinter import font, messagebox
import cv2
from dashscope import Application
from tkinter import *
from PIL import Image, ImageTk
class ChatGUI:
    def __init__(self):
        self.master = tk.Tk()
        self.master.title("人机对话系统 2025-03-25")
        self.master.geometry("1000x600")

        self.custom_font = font.Font(family="Helvetica", size=12)
        self.cap =  cv2.VideoCapture(0)
        self.image_container =None
        self.is_recording = False
        self.video_writer = None
        self.video_canvas = None
        self.dialogue_log = []
        self.camera_img = None
        self.setup_main_layout()
        self.setup_chat_area()
        self.setup_video_area()
        self.setup_button_area()
        self.update_frame()




    def setup_main_layout(self):
        """创建主容器分区"""
        # 左右分栏布局
        self.left_frame = tk.Frame(self.master, bg="#90EE90")  # 浅绿色
        self.right_frame = tk.Frame(self.master, bg="#E0FFFF")  # 浅青色

        # 使用Grid布局管理器
        self.left_frame.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        self.right_frame.grid(row=0, column=1, sticky="nsew", padx=5, pady=5)

        # 配置网格权重
        self.master.grid_columnconfigure(0, weight=1)
        self.master.grid_columnconfigure(1, weight=2)
        self.master.grid_rowconfigure(0, weight=1)

    def setup_chat_area(self):
        """配置左侧聊天区域"""
        # AI标识
        self.canvas = tk.Canvas(self.left_frame, width=60, height=60,
                                bg="#90EE90", highlightthickness=0)
        self.canvas.create_oval(10, 10, 50, 50, fill="#800080")  # 紫色圆形
        self.canvas.create_text(30, 30, text="AI", fill="#FFC0CB",
                                font=("Arial", 16, "bold"))

        # 对话内容显示
        self.chat_text = tk.Text(self.left_frame, width=40, height=20,
                                 bg="#90EE90", fg="#FF69B4",  # 粉红色
                                 font=self.custom_font, wrap=tk.WORD)
        self.chat_text.tag_configure("separator", foreground="black", spacing3=1)

        # 插入示例对话
        self.chat_text.insert(tk.END, "AI: Hello, what can I do for you.")
        self.chat_text.insert(tk.END, "\n――――――――――――――\n", "separator")
        self.chat_text.insert(tk.END, "User: what is Apple?")
        self.chat_text.insert(tk.END, "\n――――――――――――――\n", "separator")
        self.chat_text.config(state=tk.DISABLED)  # 设为只读

        # 布局管理
        self.canvas.pack(side=tk.LEFT, padx=10)
        self.chat_text.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

    def setup_video_area(self):
        """配置右侧视频采集区"""
        # 设置背景颜色为白色
        video_frame = tk.Frame(self.right_frame, bg="white")
        video_frame.pack(pady=20, padx=20, fill=tk.BOTH, expand=True)

        # 创建画布用于显示视频流
        image_width = 400
        image_height = 400
        self.video_canvas = Canvas(video_frame, bg='white', width=image_width, height=image_height)  # 绘制画布
        self.video_canvas.pack(fill=tk.BOTH, expand=True)

        # 添加文本标签说明这是一个视频区域
        # label = Label(video_frame, text='这是一个视频！', font=("黑体", 14), bg='white')
        # label.place(x=image_width / 2 - label.winfo_reqwidth() / 2, y=20)  # 使用place方法并尝试居中文本

        return video_frame

    def update_frame(self):
        # 读取一帧
        ret, frame = self.cap.read()
        frame = cv2.flip(frame, 1)
        if ret:
            # 将BGR格式转换为RGB格式
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 将帧转换为PIL图像
            im = Image.fromarray(frame)
            # 将PIL图像转换为Tkinter兼容的PhotoImage
            self.camera_img = ImageTk.PhotoImage(image=im)

            # # 更新Label的图像
            # self.video_canvas.config(image=img)
            self.video_canvas.create_image(0,0,anchor='nw',image=self.camera_img)  # 避免被垃圾回收

            # 每隔30ms更新一次帧
            self.master.after(30, self.update_frame)
        else:
            # 如果视频结束，释放资源
            self.cap.release()
            print("视频播放结束")


    def setup_button_area(self):
        """配置底部按钮区"""
        button_frame = tk.Frame(self.right_frame, bg="#808080")  # 灰色背景
        button_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=5, pady=5)

        # 按钮样式
        btn_style = {
            "bg": "#FFFF00",  # 黄色
            "fg": "red",
            "activebackground": "#CCCC00",
            "font": ("Arial", 12, "bold"),
            "width": 10,
            "height": 2
        }

        # 创建三个按钮
        self.btn_start = tk.Button(button_frame, text="开始", **btn_style, command=self.start_recording)
        self.btn_stop = tk.Button(button_frame, text="结束", **btn_style, command=self.stop_recording)
        self.btn_hold = tk.Button(button_frame, text="保留", **btn_style, command=self.save_dialogue_and_video)

        # 按钮布局
        self.btn_start.pack(side=tk.LEFT, padx=20, pady=10)
        self.btn_stop.pack(side=tk.LEFT, padx=20)
        self.btn_hold.pack(side=tk.LEFT, padx=20)

    def start_recording(self):
        if not self.is_recording:
            self.is_recording = True
            self.cap = cv2.VideoCapture(0)
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            self.video_writer = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(self.cap.get(3)), int(self.cap.get(4))))
            self.update_video_feed()
            self.btn_start.config(text="正在录制...")

    def stop_recording(self):
        if self.is_recording:
            self.is_recording = False
            self.cap.release()
            self.video_writer.release()
            self.video_label.config(text="视频采集区")
            self.btn_start.config(text="开始")

    def save_dialogue_and_video(self):
        with open("dialogue.txt", "w") as f:
            f.write("\n".join(self.dialogue_log))
        messagebox.showinfo("保存成功", "对话内容和视频已保存")

    def send_message_to_ai(self, message):
        api_key = "sk-8ac1a297ccc9487cafcf22a43bc8c4b8"
        response = Application.call(
            api_key=api_key,
            app_id='YOUR_APP_ID',  # 替换为实际的应用 ID
            prompt=message
        )

        if response.status_code != HTTPStatus.OK:
            print(f'request_id={response.request_id}')
            print(f'code={response.status_code}')
            print(f'message={response.message}')
            print(f'请参考文档：https://help.aliyun.com/zh/model-studio/developer-reference/error-code')
        else:
            ai_response = response.output.text
            self.dialogue_log.append(f"User: {message}")
            self.dialogue_log.append(f"AI: {ai_response}")
            self.display_message(message, "User")
            self.display_message(ai_response, "AI")

    def display_message(self, message, sender):
        self.chat_text.config(state=tk.NORMAL)
        self.chat_text.insert(tk.END, f"{sender}: {message}\n")
        self.chat_text.insert(tk.END, "――――――――――――――\n", "separator")
        self.chat_text.config(state=tk.DISABLED)
        self.chat_text.yview(tk.END)

if __name__ == "__main__":
    app = ChatGUI()
    app.master.mainloop()



